# HW1 Problem 1

Define the set of engineers  
$$
E = \{E_i \mid i \in \{1,2,3\}\}
$$
and the set of projects  
$$
P = \{P_j \mid j \in \{1,2,3,4\}\}.
$$

Let:
- $H_{ij}$ denote the number of hours allocated by engineer $i$ to project $j$,
- $e_{ij}$ denote the score of engineer $i$ on project $j$,
- $R_j$ denote the required hours for project $j$,
- $H_{\max}$ denote the maximum available hours for each engineer.

**Objective:**
$$
\max \sum_{i \in E} \sum_{j \in P} H_{ij} e_{ij}
$$

**Subject to:**

- Engineer hour constraints:
$$
\sum_{j \in P} H_{ij} \le H_{\max} \quad \forall i \in E
$$

- Project requirement constraints:
$$
\sum_{i \in E} H_{ij} \ge R_j \quad \forall j \in P
$$

- Non-negativity:
$$
H_{ij} \ge 0 \quad \forall i \in E,\; j \in P
$$


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import shutil
import sys
import os.path
from pyomo.environ import *

import pyomo.environ as pe
import pyomo.opt as po

In [2]:
Engg = {'E1','E2','E3'}    # Check the type by: type(Engg)

In [3]:
Proj = {'P1','P2','P3','P4'}

In [4]:
score = {
    ('E1','P1'):90,
    ('E1','P2'):80,
    ('E1','P3'):10,
    ('E1','P4'):50,
    ('E2','P1'):60,
    ('E2','P2'):70,
    ('E2','P3'):50,
    ('E2','P4'):65,
    ('E3','P1'):70,
    ('E3','P2'):40,
    ('E3','P3'):80,
    ('E3','P4'):85,
}   # Dictionary with tuples as keys (based on our defined sets)

In [5]:
hours_needed = {
    ('P1'):70,
    ('P2'):50,
    ('P3'):85,
    ('P4'):35,
}

In [6]:
max_hours = 80

In [7]:
m = pe.ConcreteModel()

In [8]:
m.Engg = pe.Set(initialize=Engg)
m.Proj = pe.Set(initialize=Proj)     # Ignore if any warnings appear

source (type: set).  This WILL potentially lead to nondeterministic behavior
in Pyomo
source (type: set).  This WILL potentially lead to nondeterministic behavior
in Pyomo


In [9]:
m.score = pe.Param(m.Engg, m.Proj, initialize=score)
m.hours_needed = pe.Param(m.Proj, initialize=hours_needed)
m.max_hours = pe.Param(initialize=max_hours)

In [10]:
m.H = pe.Var(m.Engg, m.Proj, domain=pe.NonNegativeReals) 


In [11]:
obj_expr = sum(m.H[i,j]*m.score[i,j] 
               for i in m.Engg for j in m.Proj)
m.obj = pe.Objective(sense=pe.maximize, expr=obj_expr)

In [12]:
def maxhour_rule(m,i):
    return sum(m.H[i,j] for j in m.Proj) <= m.max_hours

m.maximum_hours =  pe.Constraint(m.Engg, rule=maxhour_rule)

In [13]:
def projhour_rule(m,j):  
    return sum(m.H[i,j] for i in m.Engg) >= m.hours_needed[j]

m.proj_hours = pe.Constraint(m.Proj, rule=projhour_rule)

In [14]:
solver = po.SolverFactory('gurobi')
results = solver.solve(m, tee=True)

Read LP format model from file /var/folders/7j/jvms9gk54jj43q9b9r_pvfpr0000gn/T/tmp5vvm8wrs.pyomo.lp
Reading time = 0.00 seconds
x1: 7 rows, 12 columns, 24 nonzeros
Gurobi Optimizer version 13.0.1 build v13.0.1rc0 (mac64[arm] - Darwin 25.2.0 25C56)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 12 columns and 24 nonzeros (Max)
Model fingerprint: 0x83f61e18
Model has 12 linear objective coefficients
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 8e+01]

Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.5000000e+32   1.200000e+31   7.500000e+02      0s
       7    1.8825000e+04   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.882500000e+04


In [15]:
print("Optimal Assignment Objective: ",pe.value(m.obj))

Optimal Assignment Objective:  18825.0


In [18]:
for i in m.Engg:
    for j in m.Proj:
        if pe.value(m.H[i,j]) > 0:    
            print("Engg",i," working on Project",j,"= ",pe.value(m.H[i,j])," Hours")


Engg E3  working on Project P3 =  80.0  Hours
Engg E2  working on Project P4 =  35.0  Hours
Engg E2  working on Project P3 =  5.0  Hours
Engg E2  working on Project P2 =  40.0  Hours
Engg E1  working on Project P2 =  10.0  Hours
Engg E1  working on Project P1 =  70.0  Hours


In [19]:
for i in m.Engg:
    h = pe.value(sum(m.H[i,j] for j in m.Proj))
    print("Hours Worked by Engineer",i,"= ",h," Hours")

Hours Worked by Engineer E3 =  80.0  Hours
Hours Worked by Engineer E2 =  80.0  Hours
Hours Worked by Engineer E1 =  80.0  Hours
